In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("/workdir/")
from sktopic.models import WassersteinLatentDirichletAllocation as WLDA
from sktopic.datasets import fetch_20NewsGroups #, fetch_SearchSnippets
from sktopic.callbacks import WECoherenceScoring,TopicDiversityScoring
from octis.evaluation_metrics.classification_metrics import (
    AccuracyScore, F1Score, PrecisionScore, RecallScore,
)
from skorch.callbacks import EarlyStopping,LRScheduler,GradientNormClipping
from tqdm import tqdm 
from sktopic.utils.model_utils import get_kv, eval_all, override_word_embeddings

#from sklearn.model_selection import train_test_split
dataset = fetch_20NewsGroups()
dataset.train_test_split_stratifiedkfold()
_WECoherenceScoring = WECoherenceScoring(dataset.id2word,kv_path="/workdir/datasets/dummy_kv.txt",binary=False)
_kvs = get_kv()
_WECoherenceScoring.coherencemodel._wv = _kvs["glove.42B.300d"]
callbacks = [
        _WECoherenceScoring, #WECoherenceScoring(dataset.id2word),
        TopicDiversityScoring(dataset.id2word),
        #EarlyStopping(patience=10,threshold=1e-6),
        #LRScheduler(),
        #GradientNormClipping(gradient_clip_value=1.0)
        ]
from sktopic.models import ProductOfExpertsLatentDirichletAllocation as Model
#from sktopic.models import NeuralSinkhornTopicModel as Model

V = dataset.vocab_size
m = Model(V,20,embed_dim=300,hidden_dims=[500,500],
    #dropout_rate_theta=0.5,
    dropout_rate_hidden=0.5,
    callbacks=callbacks,
    batch_size=2000,
    max_epochs=1,
    device="cpu",
    #criterion__prior_name="dirichlet",#"gmm_ctm",
    lr=0.05,
    #topic_model=False,
    )

use_pwe = True
if use_pwe:
    from sktopic.utils.model_utils import override_word_embeddings
    m = override_word_embeddings(m,dataset,_kvs["glove.42B.300d"],train_embed=False)

m.fit(dataset.X_tr)
m.module_.decoder["decoder"].word_embeddings.weight.requires_grad 

module has word_embeddings
module.decoder['decoder'] has word_embeddings
  epoch      td       train_loss    train_ppl    valid_loss    valid_ppl    wetc_pw     dur
-------  ------  ---------------  -----------  ------------  -----------  ---------  ------
      1  0.9600  1549559612.6305    1511.1090    79340.2149    1359.1728     0.3486  1.5982


False

In [2]:
results = eval_all(m,dataset)

(1/3) Loading defaults------------------------------------
(3/3) Adding WETC-----------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]/root/.cache/pypoetry/virtualenvs/sktopic-L2WRRFYm-py3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
  2%|▏         | 1/53 [00:18<15:52, 18.33s/it]/root/.cache/pypoetry/virtualenvs/sktopic-L2WRRFYm-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  9%|▉         | 5/53 [00:27<03:44,  4.69s/it]/root/.cache/pypoetry/virtualenvs/sktopic-L2WRRFYm-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to contro

In [3]:
results

{'F1_lsvm': 0.05337647833378363,
 'Precision_lsvm': 0.0495188127467614,
 'Recall_lsvm': 0.08104570732724625,
 'ACC_lsvm': 0.08645003065603923,
 'F1_rbfsvm': 0.0309029566216528,
 'Precision_rbfsvm': 0.022853042909054423,
 'Recall_rbfsvm': 0.06942181368808947,
 'ACC_rbfsvm': 0.07510729613733906,
 'Cocherence_umass': -2.0539809455355114,
 'Cocherence_npmi': -0.01672673773629467,
 'TopicDiversity': 0.416,
 'External_InvertedRBO': 0.9745904751949681,
 'LogOddsRatio': 0.048052224314865145,
 'TopicDistributions_KLDivergence': 0.048052224314865145,
 'Internal_RBO': 0.02540952480503189,
 'KL_uniform': 0.031023531787827425,
 'KL_vacuous': 0.000880802558694588,
 'KL_background': 0.4484302437061668,
 'MeanSquaredCosineDeviatoinAmongTopics': 0.9981929063796997,
 'Purity': 0.07296696558595846,
 'NMI': 0.012337625484771084,
 'WECoherencePairwise_w2v.3M.300d': 0.122132994,
 'WECoherenceCentroid_w2v.3M.300d': -0.011238983187420473,
 'External_WordEmbeddingsInvertedRBO_w2v.3M.300d': 0.44518145020895983,